In [1]:
from data_storage import create_connection
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
connection = create_connection("../database/crypto_billionairs.db")

In [3]:
df = pd.read_sql_query('select * from ETHUSDT_5m_complete_history_long', connection)

In [4]:
def calculate_cumulated_return(df, start_sum):

    money = start_sum
    money_history = []
    sell_index = 0

    for index, row in df[['close', 'buy_indicator', 'sell_indicator']].iterrows():
        

        if row["buy_indicator"] == 1 and money != 0 and index > sell_index:
            assets = money / row["close"]
            money = 0
            buy_index = index
            sell_index = buy_index + 2015

        if sell_index == index and money == 0: 
            money = assets * df["close"][sell_index]
            money_history.append(money)
            assets = 0

        
        if len(df) - 1 == index and assets != 0:
            money = assets * row["close"]
            money_history.append(money)
            assets = 0

    return money / start_sum, money_history

result, hist = calculate_cumulated_return(df, 10000)

In [5]:
hist_diff = [j/i - 1 for i, j in zip(hist[:-1], hist[1:])]

In [67]:
#sharp ratio, sortino ratio, maximum drawdown, carl mar ratio 

In [21]:
# sharpe ratio 

#To calcualte the sharp ratio you first calculate the expected return on an investment portfolio or individual stock
# and then substract the risk-free rate of return. Then you devide that figure by then standard deviation of the portfolio
# or invstment. The shape ratio can be recalculated at the end of the year to examine the actual return rather than the expected return 
def _std_dev(data):
    # Get number of observations
    n = len(data)
    # Calculate mean
    mean = sum(data) / n
    
    # Calculate deviations from the mean
    deviations = sum([(x - mean)**2 for x in data])
    # Calculate Variance & Standard Deviation
    variance = deviations / (n - 1)
    s = variance**(1/2)
    
    return s

def sharpe_ratio(return_trading_strategy, df,  risk_free_rate=0.0):
    # Calculate Average Daily Return
    mean_return = sum(return_trading_strategy) / len(return_trading_strategy)
    # Calculate Standard Deviation
    s = _std_dev(df["return"][1:])
    # Calculate Daily Sharpe Ratio
    sharpe_ratio = (mean_return - risk_free_rate) / s
    # Annualize Daily Sharpe Ratio 
    
    return sharpe_ratio

sharpe_ratio(hist_diff, df, 0)

0.003353335788361102


31.35801861266217

In [7]:
def sortino_ratio(data, risk_free_rate=0.0):
    
    data_downside_risk = [x for x in data if x < 0]
    
    mean_return = sum(data) / len(data)
    # Calculate Standard Deviation
    s = _std_dev(data_downside_risk)
    # Calculate Daily Sharpe Ratio
    sortino_ratio = (mean_return - risk_free_rate) / s
    
    return sortino_ratio
sortino_ratio(hist_diff, 0)

1.1989109766584085

In [8]:
def max_drawdown(data):
    
    #peak = data["close"].max()
    #trough = data["close"][data["close"].idxmax():].min()
    
    peak = max(data)
    
    trough = min(data[data.index(peak):])
    
    max_drawdown = (peak - trough) / peak
    
    return max_drawdown

In [9]:
hist_return_pct = [ x / 10000 for x in hist]

In [10]:
max_drawdown(hist_return_pct)

0.0

In [ ]:
def calmar_ratio(data):
    